## Machine Translation Using Transformer Model
The application of sequence-to-sequence model and transformers have greatly enhanced the results of machine translation. In this project, I will walk through the steps needed to train a machine translation model using sequence-to-sequence model and tranformer.

In [1]:
#imprt libraries
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import math
import re
import random
import string
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset: English-French Sentence Pair
The data set used in this project comes from Tatoeba (https://tatoeba.org/en), a website where people can upload sentences in any languages and contribute thier own versions of translations in other languages. The English-French dataset used here was downloaded from https://www.manythings.org/anki/, where they preprocessed the Tatoeba dataset so that it became a text file of English Frence sentence pair. 

## Prepare data: Load file
The sentence pair is saved in datapath as a .txt file. The following function ingests the files and make them into pairs.

In [2]:
def readData(lang1, lang2, dataPath, reverse = False):
    print("Reading dataset...")
    
    #open the file and split by lines (\n)
    lines = open(dataPath, encoding = 'utf-8').read().strip().split('\n')
    
    #split lines into pairs (separated by tab, or \t) and normalize
    
    pairs = [[cleanString(s) for s in l.split('\t')] for l in lines]
    
    # Reverse if spesified
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        source_lang = LangDict(lang2)
        target_lang = LangDict(lang1)
    else:
        source_lang = LangDict(lang1)
        target_lang = LangDict(lang2)
    
    return source_lang, target_lang, pairs

## Preprocessing: Create Token-Index Dictionary 
When building neural netword models, it is common practice to map words to a number because models can only "understand" numbers". The following LangDict class is used to walk through the wholde dataset and create word-number mapping dictionary and word count dictionary.

In [3]:
SOS_token = 0
EOS_token =1

class LangDict:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"SOS", 1:"EOS"}
        self.n_words = 2 # SOS + COS = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
        

The following functions are used for preprocessing data (e.g. convert text to lowercase, remove puctuations).

In [4]:
#Helper functions

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def cleanString(s):
    #transform letters to lower case ones and remove non-letter symbols
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
Max_length = 10

exn_prefix = ("i am", "i m",
              "he is", "he s",
              "she is", "she s",
              "you are", "you re",
              "we are", "we re",
              "they are", "they re"              
             )

def filterPair(p):
    p1_tok = p[0].split(' ')
    p2_tok = p[1].split(' ')
    
    if len(p1_tok) < Max_length and len(p2_tok) < Max_length:
        return True
    else:
        return False

def BuildfilterdPairs(pairs):
    pairList = list()
    for pair in pairs:
        if filterPair(pair)==True:
            pairList.append([pair[0], pair[1]])
    return pairList

In [6]:
def prepareData(lang1, lang2, reverse = False):
    dataPath = r'./data/eng-fra.txt'
    input_lang, output_lang, pairs = readData(lang1, lang2, dataPath, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = BuildfilterdPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', False)

print("Below is an example of sentence pair:")
print(random.choice(pairs))        

Reading dataset...
Read 192341 sentence pairs
Trimmed to 136657 sentence pairs
Counting words...
Counted words:
eng 11820
fra 19411
Below is an example of sentence pair:
['they fight like cat and dog .', 'ils se disputent comme chien et chat .']


## Encoder Setup
Here we define the structure of the first sequence model, the encoder, which encodes the information of the sentence in the source language. The encoder outputs a fixed length embedding, which will serve as the input of the second sequence model, the decoder, to output prediction of word in target language. The type of layer in the sequence model family used for the encoder is to be choosed. Here, I put the GRU (gated recurrent unit) for demonstration purposes.

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device = device)

## Decoder with Attention
Decoder with attention not only takes the hidden state of the encoder as input, but also calculates the product of the current hidden status and outputs of the encoder at each step to enable the decoder to learn to "Focus its attention" on different parts of the encoder output to generate translation.

In [9]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p = 0.1, max_length = Max_length):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size*2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        ourput, hidden = self.gru(output, hidden)
        
        output = F.log_softmax(self.out(output[0]), dim =1)
        
        return output, hidden, attn_weights

    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device = device)
        

In [10]:
## Define helper functions
def indexFromSentence(lang, sentence):
    return [lang.word2index[word]  for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype = torch.long, device = device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## Model Training

In [11]:
teacherForcing_r = 0.5

#one training iteration
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, 
         decoder_optimizer, criterion, max_length = Max_length):
    
    #initialize encoder hidden layer weights
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
    
    loss = 0
    
    #encoder
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    
    teacherForcing = True if random.random() < teacherForcing_r else False
    #decoder    
    if teacherForcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                                                                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
            
            
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                                                            decoder_input, decoder_hidden, encoder_outputs)
        
        
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
        
            loss += criterion(decoder_output, target_tensor[di])
        
            if decoder_input.item() == EOS_token:
                break;
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length

In [12]:
#helper function: timers

def asMinutes(s):
    m = math.floor(s/ 60)
    s -= m * 60
    return '%dm %ds' % (m,s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
    

In [1]:
#iterate over training process
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every = 100, learning_rate = 0.01):
    
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr = learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr = learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                     for i in range(n_iters)]

    criterion = nn.NLLLoss()
    
    for iter in range(1, n_iters + 1):
        
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder,
                    decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter/ n_iters), 
                                         iter, iter/ n_iters  *  100, print_loss_avg))


In [14]:
#plot resutls
import matplotlib.pyplot as plt
plt.switch_backend('agg')

import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    #ticks at regular inter
    loc = ticker.MultipleLocator(base= 0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Evaluate the model

In [15]:
def evaluate(encoder, decoder, sentence, max_length=Max_length):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
        for ei in range(input_length):

            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
            
            
        decoder_input = torch.tensor([[SOS_token]], device = device)#Start of sentens(SOS)
        
        decoder_hidden  = encoder_hidden
        
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi, = decoder_output.data.topk(1)
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break;
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            
            decoder_input = topi.squeeze().detach()
        
        return decoded_words, decoder_attentions[:di + 1]
            

In [17]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
atten_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, atten_decoder1, 60000, print_every=5000)

14m 28s (- 159m 10s) (5000 8%) 4.4738
29m 20s (- 146m 44s) (10000 16%) 4.0735
44m 29s (- 133m 27s) (15000 25%) 3.8906
59m 39s (- 119m 18s) (20000 33%) 3.7712
74m 52s (- 104m 48s) (25000 41%) 3.7296
90m 12s (- 90m 12s) (30000 50%) 3.6366
105m 27s (- 75m 19s) (35000 58%) 3.6052
120m 39s (- 60m 19s) (40000 66%) 3.5151
135m 57s (- 45m 19s) (45000 75%) 3.4660
151m 17s (- 30m 15s) (50000 83%) 3.4280
166m 35s (- 15m 8s) (55000 91%) 3.4521
182m 9s (- 0m 0s) (60000 100%) 3.4214


In [18]:
def evaluateRandomly(encoder, decoder, n = 10):
    for i in range(n):
        pair = random.choice(pairs)
        print('Sentence from source language: ', pair[0])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('Model generated sentence: ', output_sentence)
        print('Sentence from target language: ', pair[1])
        print('')

## Examples of tranlation produced by current model

In [19]:
evaluateRandomly(encoder1, atten_decoder1)

Sentence from source language:  you ve betrayed us .
Model generated sentence:  vous etes au est . <EOS>
Sentence from target language:  tu nous as trahies .

Sentence from source language:  you re the best singer i know .
Model generated sentence:  vous etes la meilleure . . . <EOS>
Sentence from target language:  vous etes le meilleur chanteur que je connaisse .

Sentence from source language:  it just came out .
Model generated sentence:  c est juste de . <EOS>
Sentence from target language:  il vient de s avouer homo .

Sentence from source language:  this town has narrow streets .
Model generated sentence:  ce a a a de la . <EOS>
Sentence from target language:  cette ville a des rues etroites .

Sentence from source language:  how s your project coming along ?
Model generated sentence:  comment va ta . <EOS>
Sentence from target language:  comment progresse ton projet ?

Sentence from source language:  you d better go to bed .
Model generated sentence:  vous mieux mieux de au lit 

## References

1. [NLP FROM SCRATCH: TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK AND ATTENTION](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html), PyTorch tutorial by Sean  Robertson.
2. [Aladdin Persson's Github](https://github.com/aladdinpersson)
3. Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Fethi Bougares, Holger Schwenk, and Yoshua Bengio. Learning phrase representations using rnn encoder-decoder for statistical machine translation. CoRR, abs/1406.1078, 2014.
